## Z3 Semantic Parser

### HaluEval Knowledge-Grounded Q/A dataset, 10,000 hallucination/correct examples
### https://github.com/RUCAIBox/HaluEval/blob/main/data/qa_data.json


#### Example Input Format: 
...
{"knowledge": "Margaret \"Peggy\" Seeger (born June 17, 1935) is an American folksinger. She is also well known in Britain, where she has lived for more than 30 years, and was married to the singer and songwriter Ewan MacColl until his death in 1989.James Henry Miller (25 January 1915 – 22 October 1989), better known by his stage name Ewan MacColl, was an English folk singer, songwriter, communist, labour activist, actor, poet, playwright and record producer.", "question": " What nationality was James Henry Miller's wife?", 
...

#### Example Output Format: 

...

answer: "James Henry Miller's wife was British."

z3:
```
# Predicates
is_folksinger = Function('is_folksinger', StringSort(), BoolSort())
is_wife_of = Function('is_wife_of', StringSort(), StringSort(), BoolSort())
nationality = Function('nationality', StringSort(), StringSort(), BoolSort())
known_in = Function('known_in', StringSort(), StringSort(), BoolSort())
lived_for_years = Function('lived_for_years', StringSort(), IntSort(), BoolSort())

# Constants
peggy_seeger = StringVal('Peggy Seeger')
james_henry_miller = StringVal('James Henry Miller')
usa = StringVal('USA')
britain = StringVal('Britain')
british = StringVal('British')
more_than_30 = IntVal(30)

# Axioms
axioms = [
    is_folksinger(peggy_seeger),
    is_wife_of(peggy_seeger, james_henry_miller),
    known_in(peggy_seeger, britain),
    lived_for_years(peggy_seeger, more_than_30),
    nationality(peggy_seeger, usa)
]

# Query based on Answer
query = nationality(peggy_seeger, british)
```
...


### HaluEval Knowledge-Grounded Dialogue dataset, 10,000 hallucination/correct examples
### https://github.com/RUCAIBox/HaluEval/blob/main/data/dialogue_data.json

#### Example of Input Format:
knowledge_question:
...
{"knowledge": "Iron Man is starring Robert Downey Jr.Robert Downey Jr. starred in Zodiac (Crime Fiction Film)Zodiac (Crime Fiction Film) is starring Jake Gyllenhaal", "dialogue_history": "[Human]: Do you like Iron Man [Assistant]: Sure do! Robert Downey Jr. is a favorite. [Human]: Yes i like him too did you know he also was in Zodiac a crime fiction film. ", "
...

#### Example Output Format:
answer: "I'm not a fan of crime movies, but I did know that RDJ starred in Zodiac with Tom Hanks."

z3:
```
# Predicates
is_starring = Function('is_starring', StringSort(), StringSort(), BoolSort())
starred_in = Function('starred_in', StringSort(), StringSort(), BoolSort())

# Constants
iron_man = StringVal('Iron Man')
robert_downey_jr = StringVal('Robert Downey Jr.')
zodiac = StringVal('Zodiac (Crime Fiction Film)')
jake_gyllenhaal = StringVal('Jake Gyllenhaal')
tom_hanks = StringVal('Tom Hanks')

# Axioms
axioms = [
    is_starring(iron_man, robert_downey_jr),
    starred_in(robert_downey_jr, zodiac),
    is_starring(zodiac, jake_gyllenhaal)
]

# Query based on Answer
query = is_starring(zodiac, tom_hanks)
```

In [13]:
from z3 import *

In [14]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

from openai import OpenAI

In [15]:
client = OpenAI(api_key="")

In [18]:
def clean_text(text):
    """
    Removes the patterns ':   ```' from the start and ' ```' from the end of a string,
    while preserving the actual content and quotes.
    
    Args:
        text (str): Input text containing the patterns to remove
        
    Returns:
        str: Cleaned text with patterns removed
    """
    # Remove the starting pattern
    if text.startswith('":'):
        text = text[2:]
    
    # Remove the ending pattern
    if text.endswith('"'):
        text = text[:-3]
    
    else:
        text = text[:-2]
        
    return text.strip()

def clean_code_snippet(snippet):
    """
    Clean a code snippet by removing triple backticks and language identifiers.
    
    Args:
        snippet (str): Raw code snippet
        
    Returns:
        str: Cleaned code snippet
    """
    # Remove triple backticks and language identifier
    cleaned = snippet.replace('```python\n', '').replace('```z3\n', '').replace('```', '')
    # Replace escaped newlines with actual newlines
    return cleaned.replace('\\n', '\n')

def write_snippets_to_file(iteration, snippets, filename="formatted_snippets_dialogue_syntax_error.txt"):
    """
    Write formatted code snippets to a file.
    
    Args:
        iteration (int): Current iteration number
        snippets (dict): Dictionary containing the snippets
        filename (str): Name of the output file
    """
    mode = 'a' if iteration > 0 else 'w'  
    with open(filename, 'a') as f:
        f.write(f"\nIteration {iteration}:\n")
        f.write("="*50 + "\n")
        
        for snippet_type, code in snippets.items():
            f.write(f"\n{snippet_type}:\n")
            f.write(clean_code_snippet(code))
            f.write("\n" + "-"*40 + "\n")

results = {}
iteration = 0
with open("halu_dialogue_syntax_error.txt", 'r') as f:
    for i in f:
        # Split knowledge, question, and answers
        knowledge_question = i.split("right_response")[0]
        right_answer = i.split("right_response")[1].split("hallucinated_response")[0]
        right_answer = clean_text(right_answer)
        hallucinated_answer = i.split("right_response")[1].split("hallucinated_response")[1]
        hallucinated_answer = clean_text(hallucinated_answer)
        if iteration not in results:
            results[iteration] = {}
        completion_2 = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user", 
                "content": (
                    "Lets take this step by step, Your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is an example expected input with a correct answer: "
                    "Question: {\"knowledge\": \"Spongebob Squarepants is a cartoon television series developed for Nickelodeon based on aquatic sea creatures. It first aired in May 1999. \"dialogue_history\": \"[Assistant] The television series on Nickelodeon is Spongebob Squarepants [Human] it first aired in what year?\", "
                    "Correct Answer: 1999"

                    "Here is the expected output: "
                    "# Predicates"
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort())"
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort())"
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort())"
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort())"

                    "# Constants"
                    "spongebob_squarepants = StringVal('Spongebob Squarepants')"
                    "nickelodeon = StringVal('Nickelodeon')"
                    "aquatic_creatures = StringVal('Aquatic Creatures')"

                    "# Axioms"
                    "axioms = ["
                    "    is_tv_series(spongebob_squarepants),"
                    "    developed_for(spongebob_squarepants, nickelodeon),"
                    "    based_on(spongebob_squarepants, aquatic_creatures),"
                    "    first_aired_in(spongebob_squarepants, 1999)"
                    "]"

                    "# Query based on Answer"
                    "query = first_aired_in(spongebob, 1999)"
                    
                    "Here is an example expected input with an incorrect answer: "
                    "Question: {\"knowledge\": \"Spongebob Squarepants is a cartoon television series developed for Nickelodeon based on aquatic sea creatures. It first aired in May 1999. \"dialogue_history\": \"[Assistant] The television series on Nickelodeon is Spongebob Squarepants [Human] it first aired in what year?\", "
                    "Incorrect Answer: 2003"

                    "Here is the expected output: "
                    "# Predicates"
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort())"
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort())"
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort())"
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort())"

                    "# Constants"
                    "spongebob_squarepants = StringVal('Spongebob Squarepants')"
                    "nickelodeon = StringVal('Nickelodeon')"
                    "aquatic_creatures = StringVal('Aquatic Creatures')"

                    "# Axioms"
                    "axioms = ["
                    "    is_tv_series(spongebob_squarepants),"
                    "    developed_for(spongebob_squarepants, nickelodeon),"
                    "    based_on(spongebob_squarepants, aquatic_creatures),"
                    "    first_aired_in(spongebob_squarepants, 1999)"
                    "]"

                    "# Query based on Answer"
                    "query = first_aired_in(spongebob_squarepants, 2003)"
                    
                    "Dont use a string literal directly in the Z3 expression. "
                    "In Z3, you need to convert string literals to Z3 StringVal objects before using them in expressions. "
                    "The error could have occurred due to an 'Invalid code block: missing axioms or query' make sure to have "
                    "appropriate axioms to be tested in the query in Z3 syntax. "
                    "Be consistent to what is stated in the input, regardless of any logical inconsistencies, you are to strictly follow what is stated in the input knowledge, question, and answer and translate this directly to Z3."
                    "Even if the Z3 query will fail you are to be consistent with the knowledge, question, and answer given to you and translate this directly to Z3 syntax"
#                     "I will also supply a Z3 Error to avoid and ensure is fixed. "
                    "Now convert this sentence to Z3 syntax with a query based on the Answer, ONLY OUTPUT Z3 NOTHING ELSE: " +    
                    "Question with Z3 Error: " + knowledge_question + " Answer: " + right_answer
                )
            }])
            
        results[iteration]["right_answer"] = completion_2.choices[0].message.content
        
        completion_3 = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user", 
                "content": (
                    "Lets take this step by step, Your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is an example expected input with a correct answer: "
                    "Question: {\"knowledge\": \"Spongebob Squarepants is a cartoon television series developed for Nickelodeon based on aquatic sea creatures. It first aired in May 1999. \"dialogue_history\": \"[Assistant] A cartoon television series example is Spongebob Squarepants [Human] Spongebob was developed for what TV Network?\", "
                    "Correct Answer: Nickelodeon"

                    "Here is the expected output: "
                    "# Predicates"
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort())"
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort())"
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort())"
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort())"

                    "# Constants"
                    "spongebob_squarepants = StringVal('Spongebob Squarepants')"
                    "nickelodeon = StringVal('Nickelodeon')"
                    "aquatic_creatures = StringVal('Aquatic Creatures')"

                    "# Axioms"
                    "axioms = ["
                    "    is_tv_series(spongebob_squarepants),"
                    "    developed_for(spongebob_squarepants, nickelodeon),"
                    "    based_on(spongebob_squarepants, aquatic_creatures),"
                    "    first_aired_in(spongebob_squarepants, 1999)"
                    "]"

                    "# Query based on Answer"
                    "query = developed_for(spongebob_squarepants, nickelodeon)"
                    
                    "Here is an example expected input with an incorrect answer: "
                    "Question: {\"knowledge\": \"Spongebob Squarepants is a cartoon television series developed for Nickelodeon based on aquatic sea creatures. It first aired in May 1999. \"dialogue_history\": \"[Assistant] A cartoon television series example is Spongebob Squarepants [Human] Spongebob was developed for what TV Network?\", "
                    "Incorrect Answer: It was developed for Cartoon Network"

                    "Here is the expected output: "
                    "# Predicates"
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort())"
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort())"
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort())"
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort())"

                    "# Constants"
                    "spongebob_squarepants = StringVal('Spongebob Squarepants')"
                    "nickelodeon = StringVal('Nickelodeon')"
                    "cartoon_network = StringVal('Cartoon Network')"
                    "aquatic_creatures = StringVal('Aquatic Creatures')"

                    "# Axioms"
                    "axioms = ["
                    "    is_tv_series(spongebob_squarepants),"
                    "    developed_for(spongebob_squarepants, nickelodeon),"
                    "    based_on(spongebob_squarepants, aquatic_creatures),"
                    "    first_aired_in(spongebob_squarepants, 1999)"
                    "]"

                    "# Query based on Answer"
                    "query = developed_for(spongebob_squarepants, cartoon_network)"
                    
                    "Dont use a string literal directly in the Z3 expression. "
                    "In Z3, you need to convert string literals to Z3 StringVal objects before using them in expressions. "
                    "The error could have occurred due to an 'Invalid code block: missing axioms or query' make sure to have "
                    "appropriate axioms to be tested in the query in Z3 syntax. "
                    "Be consistent to what is stated in the input, regardless of any logical inconsistencies, you are to strictly follow what is stated in the input knowledge, question, and answer and translate this directly to Z3."
                    "Even if the Z3 query will fail you are to be consistent with the knowledge, question, and answer given to you and translate this directly to Z3 syntax"
#                     "I will also supply a Z3 Error to avoid and ensure is fixed. "
                    "Now convert this sentence to Z3 syntax with a query based on the Answer, ONLY OUTPUT Z3 NOTHING ELSE: " +  
                    "Question with Z3 Error: " + knowledge_question + " Answer: " + hallucinated_answer
                )
            }])
            
        results[iteration]["hallucinated_answer"] = completion_3.choices[0].message.content
        
        # Clean and write formatted snippets
        current_snippets = {
            "knowledge_question": knowledge_question,
            "right_answer": right_answer,
            "right_answer_z3": results[iteration]["right_answer"],
            "hallucinated_answer": hallucinated_answer,
            "hallucinated_answer_z3": results[iteration]["hallucinated_answer"]
        }
        write_snippets_to_file(iteration, current_snippets)
        
        #print(iteration)
        iteration += 1

In [21]:
results = {}
iteration = 0
with open("/workspace/HLINC/data/halu_dialogue.txt", 'r') as f:
    for i in f:
        # Split knowledge, question, and answers
        knowledge_question = i.split("right_response")[0]
        right_answer = i.split("right_response")[1].split("hallucinated_response")[0]
        right_answer = clean_text(right_answer)
        hallucinated_answer = i.split("right_response")[1].split("hallucinated_response")[1]
        hallucinated_answer = clean_text(hallucinated_answer)
        if iteration not in results:
            results[iteration] = {}

        results[iteration]["right_answer"] = ""

        completion_3 = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user", 
                "content": (
                    "You are going to be given a Knowledge grounded question and answer. "
                    "Your job is to output 'Correct' or 'Incorrect' based on what you think the correct answer is. "
                    "DO NOT OUTPUT ANYTHING OTHER THAN 'Correct' or 'Incorrect' "
                    "Question: " + knowledge_question + " Answer: " + hallucinated_answer
                )
            }])
            
        results[iteration]["hallucinated_answer"] = completion_3.choices[0].message.content
        
        # Clean and write formatted snippets
        current_snippets = {
            "knowledge_question": knowledge_question,
            "right_answer": right_answer,
            "right_answer_z3": results[iteration]["right_answer"],
            "hallucinated_answer": hallucinated_answer,
            "hallucinated_answer_z3": results[iteration]["hallucinated_answer"]
        }
        write_snippets_to_file(iteration, current_snippets, "formatted_snippets_dialogue_chatgpt.txt")
        
        #print(iteration)
        iteration += 1